### Authenticate and get headers

In [ ]:
import requests
from _includes import config
from _includes import helper

config_dict = config.config_dict

api = helper.APIHelper(config_dict)
base_site_url = f'{api.server}/api/{api.api_version}/sites/{api.site_id}'
base_api_url = f'{api.server}/api/-'
headers = api.headers

### List Existing Definitions

In [ ]:
endpoint = f'{base_api_url}/pulse/definitions'

metadata_list = []
api_response = requests.get(endpoint, headers=headers)
api_response_json = api_response.json()
metadata_list.extend(api_response_json['definitions'])

# Handle the Pagination
while 'next_page_token' in api_response_json and api_response_json['next_page_token'] != '':
    next_page_token = api_response_json['next_page_token']
    api_response = requests.get(f'{endpoint}?pageToken={next_page_token}', headers=headers)
    api_response_json = api_response.json()
    metadata_list.extend(api_response_json['definitions'])

# Print Sample Data
for metadata in metadata_list:
    print(metadata)

### Transforming the JSON data into a structured format to make it more manageable

In [ ]:
import pandas as pd
import json

definition_list = []

for item in metadata_list:
    metadata = item.get('metadata', {})
    specification = item.get('specification', {})
    basic_specification = specification.get('basic_specification', {})
    measure = basic_specification.get('measure', {})
    time_dimension = basic_specification.get('time_dimension', {})
    filters = basic_specification.get('filters', [])
    extension_options = item.get('extension_options', {})
    representation_options = item.get('representation_options', {})
    viz_state_specification = specification.get('viz_state_specification', {})
    viz_state_string = viz_state_specification.get('viz_state_string', '{}')
    formula = ''
    formula_time_column = []

    try:
        viz_state_json = json.loads(viz_state_string)
        formula = viz_state_json.get('dataModel', {}).get('dataSource', [])[0].get('columnsToAdd', [])[0].get('calc', {}).get('formula', '')
        # formula_time_column = viz_state_json.get('vizState', {}).get('columns', [])[0].get('fieldOnShelf', {}).get('component', [])
        formula_time_column = viz_state_json.get('vizState', {}).get('columns', [])[0].get('fieldCaption', '')
    except (json.JSONDecodeError, IndexError, KeyError):
        pass

    definition_list.append({
        'definition_name': metadata.get('name', ''),
        'definition_id': metadata.get('id', ''),
        'datasource_id': specification.get('datasource', {}).get('id', ''),
        'measure_field': measure.get('field', ''),
        'measure_aggregation': measure.get('aggregation', ''),
        'time_dimension_field': time_dimension.get('field', ''),
        'formula': formula,
        'formula_time_dimension': formula_time_column,
        'filters': filters,
        'is_running_total': basic_specification.get('is_running_total', False),
        'allowed_dimensions': extension_options.get('allowed_dimensions', []),
        'sentiment_type': representation_options.get('sentiment_type', ''),
        'allowed_granularities': extension_options.get('allowed_granularities', []),
    })

# Create DataFrame
df = pd.DataFrame(definition_list)
df